In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *


In [2]:
spark=SparkSession.builder.appName('desafio-python').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/29 18:26:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        white-space: pre !important;
    }
    pre { white-space: pre !important; }
</style>
""")

In [4]:
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.repl.eagerEval.truncate', 30)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows', 5)

In [5]:
# dataframes_dict = {}
# dataframes_dict['metadata_de_facturas']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/bkpf_vbrk.parquet")
# dataframes_dict['facturas_por_cliente']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/vbpa_kna1.parquet")
# dataframes_dict['materiales']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/vbrp_mara_marm.parquet")
# dataframes_dict['paises']=spark.read.parquet("/Users/jeikorodriguez/Desktop/curso python/T005T.parquet")

########################################################################################################################################

dataframes_dict = {}
dataframes_dict['metadata_de_facturas']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/bkpf_vbrk.parquet")
dataframes_dict['facturas_por_cliente']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/vbpa_kna1.parquet")
dataframes_dict['materiales']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/vbrp_mara_marm.parquet")
dataframes_dict['paises']=spark.read.parquet("/Users/tamara/Desktop/Principal/KRANIO/DESAFIO-PYTHON/dataset/T005T.parquet")

In [6]:
dataframes_dict['materiales'].show(15, False)


22/08/29 18:26:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+--------+----------+--------+------------+-------------+--------+-------+-------------+--------+---------+----------+--------+---------+----------+---------------+---------------+---------+---------------------------------------+---------------+----------+---------------+-----------------+------------+------+------+----+----------+-------------+------------+------------+-------------+--------------+-------------+--------------+-------+-------------+-------+---------------+--------+---------------+---------------------------+---------------+---------------+-------------+-------------+---------+----------+------------+--------------+-------------+-----------+-------------+----+--------------+------------+------------+---------+-------------+--------------+--------------+----------+---------+--------------+---------------+-------------+--------------+-------------+-------------+--------------+--------+--------+-----+------+------+-------------+-------------+-----------

#### Filtros por Mandante = 400

In [7]:
df_metadata_facturas = dataframes_dict['metadata_de_facturas'].where(col('Mandante') == '400')

In [8]:
df_facturas_por_cliente = dataframes_dict['facturas_por_cliente'].where(col('Mandante') == '400')

In [9]:
df_materiales = dataframes_dict['materiales'].where(col('Mandante') == '400')

In [10]:
df_paises = dataframes_dict['paises'].where(col('Mandante') == '400').where(col('Idioma') == 'S')
# Ocupar sólo "S" Y info mandante 400

## SELECT DE TABLAS

In [11]:
df_metadata_facturas = df_metadata_facturas.select('Mandante', 'Factura', 'Clase_factura',).where(col('Factura').isNotNull())


In [12]:
df_facturas_por_cliente = df_facturas_por_cliente.select('Mandante','Factura','Posicion_de_Factura','Pais', 'Cliente_Solicitante', 'Cliente_Destinatario_de_Mercancia')

In [13]:
df_materiales = df_materiales.groupBy('Factura', 'Material', 'Unidad_de_medida_Volumen_Destino', 'Pais').agg(sum(col('Volumen')))

In [14]:
df_paises = df_paises.select('Mandante', 'Idioma', 'Pais' , 'Pais_descr')

In [15]:
# # df_metadata_facturas.show(2, False)
# # df_facturas_por_cliente.show(2, False)
# df_materiales.show(2, False)
# df_paises.show(2, False)

#### Join

In [16]:
def join_cliente_clase_factura(dataframes_dict):
    df_result = df_facturas_por_cliente.join(df_metadata_facturas, 'Factura', 'left').drop(df_metadata_facturas.Mandante)

    return {'add_join_cliente_clase_factura': df_result}

def join_materiales_factura_x_clase_factura(dataframes_dict):
    df_clase_factura = dataframes_dict['add_join_cliente_clase_factura']
    df_result = df_materiales.join(df_clase_factura, 'Factura', 'left').drop(df_clase_factura.Pais).drop(df_clase_factura.Mandante)
    print(dataframes_dict['materiales'].count())
    print(df_clase_factura.count())
    print(df_result.count())

    return {'add_join_materiales_factura_x_clase_factura': df_result}

In [17]:
def gen_srv_desafio_python(dataframes_dict):

    functions_list = [
        join_cliente_clase_factura,
        join_materiales_factura_x_clase_factura,
        # Otras transformaciones
    ]

    for function in functions_list:
        print(function)
        result_element = function(dataframes_dict)
        dataframes_dict.update(result_element)

    return dataframes_dict['add_join_materiales_factura_x_clase_factura'] # Modificar al último dataframe retornado

In [18]:
test = gen_srv_desafio_python(dataframes_dict)

<function join_cliente_clase_factura at 0x1068e5ea0>
<function join_materiales_factura_x_clase_factura at 0x1068e5f30>
80708
19135
43098


In [19]:
test

Factura,Material,Unidad_de_medida_Volumen_Destino,Pais,sum(Volumen),Posicion_de_Factura,Cliente_Solicitante,Cliente_Destinatario_de_Mercancia,Clase_factura
0150140322,M300BLK000192,M3,CL,8.776404290129069,0,48352,null,YPVE
0150140322,M300BLK000192,M3,CL,8.776404290129069,0,null,null,YPVE
0150140322,M300BLK000192,M3,CL,8.776404290129069,0,null,null,YPVE
0150140322,M300BLK000192,M3,CL,8.776404290129069,0,null,null,YPVE
0150140322,M300BLK000192,M3,CL,8.776404290129069,10,null,null,YPVE


In [20]:
dataframes_dict['materiales'].count()

80708